In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from keras_model import *

In [61]:
main()

Epoch 1/50


/opt/anaconda3/envs/tensorflow_env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.2625 - loss: 2.3106 - val_accuracy: 0.3783 - val_loss: 1.6598
Epoch 2/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.4185 - loss: 1.5459 - val_accuracy: 0.4735 - val_loss: 1.3513
Epoch 3/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.5045 - loss: 1.3192 - val_accuracy: 0.2732 - val_loss: 2.0792
Epoch 4/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.3312 - loss: 1.8327 - val_accuracy: 0.4408 - val_loss: 1.4722
Epoch 5/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.3531 - loss: 1.8373 - val_accuracy: 0.3596 - val_loss: 1.6677
Epoch 6/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.4081 - loss: 1.5852 - val_accuracy: 0.4368 - val_loss: 1.4466
Epoch 7/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.3712 - loss: 1.6842 - val_accuracy: 0.2923 - val_loss: 1.9076
252/252 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.4823 - loss: 1.3343
Validation Loss: 1.351271